## The $\lambda$-invariant of the Iwasawa power series for Dirichlet characters via interpolation of Bernoulli numbers


### Preprint *On Iwasawa $\lambda$-invariants for abelian number fields and random matrix heuristics* by Daniel Delbourgo and Heiko Knospe

In [1]:
from sage.modular.dirichlet import DirichletCharacter

In [2]:
def power_series_interpolation(chi,p,i):
    r"""
    Returns the coefficients of the Iwasawa power series.
    The computation uses interpolaton of Bernoulli numbers.
    
    INPUT:
    
  
    - ``chi``-- Dirichlet character, uses the SageMath class DirichletCharacter.
    
    - ``p`` -- prime number.
    
    - ``i`` -- Teichmuller twist of $\chi$.
    
    
    OUTPUT: array of coefficients of the Iwasawa power series, starting with the constant coefficient
    
    
    """
    n = chi.order() 
    chi0 = chi.change_ring(CyclotomicField(n)) 
    
    f = mod(p,n).multiplicative_order() # degree of field extension of Qp containing $n$-th root of unity
    
    L.<pi>=Qq(p^f) # unramified field extension of Qp
    
    zeta=L.roots_of_unity(n)[1]
    chip=chi0(p).lift()(zeta)
    
    points=[(L((1+p)^(j-1)-1),(-1)*chi0.bernoulli(j).lift()(zeta)/j * (1-chip*p^(j-1))) for j in range(p-1+i,15*p,p-1)]
    f = PolynomialRing(L,'x').lagrange_polynomial(points)
    
    coeff=list(f)
    return(coeff)

In [3]:
def lambda_invariant(chi,p,i):
    r"""
    Returns the lambda-coefficient of the Iwasawa power series of $\chi \omega^i$.
    The computation uses interpolaton of Bernoulli numbers.
    
    INPUT:
    
  
    - ``chi``-- Dirichlet character, uses the SageMath class DirichletCharacter.
    
    - ``p`` -- prime number.
    
    - ``i`` -- Teichmuller twist of $\chi$.
    
    
    OUTPUT: lambda-invariant
    
    
    """
    
    n = chi.order() # order of $\chi$
    
    if gcd(n,p)>1:
        print("Not yet implemented.")
        return()
    
    cond = chi.conductor() # conductor of $\chi$
    
    if gcd(cond,p)>1:
        print("Not yet implemented.")
        return()
    
    f = mod(p,n).multiplicative_order()
    
    L.<pi>=Qq(p^f) # unramified field extension of Qp
    
    coeffs=power_series_interpolation(chi,p,i)

    v = L.valuation()
    j=0
    while j<15:
        if v(coeffs[j])<=0: 
            break;
        else:
            j+=1
    return(j)  # lambda-invariant       

In [4]:
# Example: conductor 19, p=7, twist=1
H = DirichletGroup(19, base_ring=CyclotomicField(6))
M = H._module
chi = DirichletCharacter(H, M([1]))
lambda_invariant(chi,7,1) # lambda=4, same result as with other method power_series_dirichlet

4

In [5]:
def lambda_quad_even(p,N,M):
    
    r"""
    Returns the lambda coefficients for even quadratic characters.
    
    INPUT:
    
    
    - ``p`` -- prime number.
    
    - ``n`` -- order of character
    
    - ``N`` -- lower bound for the conductor
    
    - ``M`` -- upper bound for the conductor
    
    
    OUTPUT: array of occurences of lambda-invariants of the Iwasawa power series, starting with $\lambda=0$
    
    
    """
    
    K=Qp(p)
    Ltot=vector(ZZ,15)
    for twist in range(0,p-1,2):
        print("p=",p,"twist=",twist)
        L=[0]*15
        for n in range(N,M):
            H = DirichletGroup(n, base_ring=CyclotomicField(2))
            for i in range(H.order()):
                if H[i].order()==2 and H[i].is_even()==True and H[i].is_primitive() and gcd(H[i].conductor(),p)==1:
               
                    coeffs=power_series_interpolation(H[i],p,twist)

                    v = K.valuation()
                    j=0
                    while j<15:
                        if v(coeffs[j])<=0: 
                            L[j]+=1
                            #if j>0:
                            print("lambda=",j," ", H[i])
                            break;
                        else:
                            j+=1
    
        print("Total number=",sum(L))
        if sum(L)>0:
            Lprop=[RR(x/sum(L)).n(digits=4) for x in L]
            print("Proportions=",Lprop)
        Ltot=Ltot+vector(L)    
    return(Ltot)

In [6]:
# Example: p=3, quadratic even characters of conductor<100
lambda_quad_even(3,1,100)

p= 3 twist= 0
lambda= 0   Dirichlet character modulo 5 of conductor 5 mapping 2 |--> -1
lambda= 0   Dirichlet character modulo 8 of conductor 8 mapping 7 |--> 1, 5 |--> -1
lambda= 0   Dirichlet character modulo 13 of conductor 13 mapping 2 |--> -1
lambda= 0   Dirichlet character modulo 17 of conductor 17 mapping 3 |--> -1
lambda= 0   Dirichlet character modulo 28 of conductor 28 mapping 15 |--> -1, 17 |--> -1
lambda= 1   Dirichlet character modulo 29 of conductor 29 mapping 2 |--> -1
lambda= 0   Dirichlet character modulo 37 of conductor 37 mapping 2 |--> -1
lambda= 0   Dirichlet character modulo 40 of conductor 40 mapping 31 |--> 1, 21 |--> -1, 17 |--> -1
lambda= 0   Dirichlet character modulo 41 of conductor 41 mapping 6 |--> -1
lambda= 0   Dirichlet character modulo 44 of conductor 44 mapping 23 |--> -1, 13 |--> -1
lambda= 0   Dirichlet character modulo 53 of conductor 53 mapping 2 |--> -1
lambda= 0   Dirichlet character modulo 56 of conductor 56 mapping 15 |--> -1, 29 |--> -1, 17 |

(19, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [7]:
def lambda_even(p,n,N,M):
    
    r"""
    Returns the lambda coefficients for even characters of a given order.
    
    INPUT:
    
    
    - ``p`` -- prime number.
    
    - ``n`` -- order of character
    
    - ``N`` -- lower bound for the conductor
    
    - ``M`` -- upper bound for the conductor
    
    
    OUTPUT: array of occurences of lambda-invariants of the Iwasawa power series, starting with $\lambda=0$
    
    
    """
    
    f = mod(p,n).multiplicative_order() # degree of field extension of Qp containing $n$-th root of unity
    
    K.<pi>=Qq(p^f) # unramified field extension of Qp
      
    Ltot=vector(ZZ,15)
    for twist in range(0,p-1,2):
        print("p=",p,"twist=",twist)
        L=[0]*15
        for k in range(N,M):
            H = DirichletGroup(k, base_ring=CyclotomicField(n))
            for i in range(H.order()):
                
                if H[i].order()==n and H[i].is_even()==True and H[i].is_primitive() and gcd(H[i].conductor(),p)==1:
               
                    coeffs=power_series_interpolation(H[i],p,twist)

                    v = K.valuation()
                    j=0
                    while j<15:
                        if v(coeffs[j])<=0: 
                            L[j]+=1
                            #if j>0:
                            print("lambda=",j," ", H[i])
                            break;
                        else:
                            j+=1
    
        print("Total number=",sum(L))
        if sum(L)>0:
            Lprop=[RR(x/sum(L)).n(digits=4) for x in L]
            print("Proportions=",Lprop)
        Ltot=Ltot+vector(L)    
    return(Ltot)

In [8]:
# Example: p=7, cubic characters of conductor <100
lambda_even(7,3,1,100)

p= 7 twist= 0
lambda= 2   Dirichlet character modulo 9 of conductor 9 mapping 2 |--> zeta3
lambda= 0   Dirichlet character modulo 9 of conductor 9 mapping 2 |--> -zeta3 - 1
lambda= 0   Dirichlet character modulo 13 of conductor 13 mapping 2 |--> zeta3
lambda= 0   Dirichlet character modulo 13 of conductor 13 mapping 2 |--> -zeta3 - 1
lambda= 0   Dirichlet character modulo 19 of conductor 19 mapping 2 |--> zeta3
lambda= 0   Dirichlet character modulo 19 of conductor 19 mapping 2 |--> -zeta3 - 1
lambda= 0   Dirichlet character modulo 31 of conductor 31 mapping 3 |--> zeta3
lambda= 0   Dirichlet character modulo 31 of conductor 31 mapping 3 |--> -zeta3 - 1
lambda= 0   Dirichlet character modulo 37 of conductor 37 mapping 2 |--> zeta3
lambda= 0   Dirichlet character modulo 37 of conductor 37 mapping 2 |--> -zeta3 - 1
lambda= 0   Dirichlet character modulo 43 of conductor 43 mapping 3 |--> zeta3
lambda= 0   Dirichlet character modulo 43 of conductor 43 mapping 3 |--> -zeta3 - 1
lambda= 0   

(59, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

### List of characters and twists of conductor $<100$, $p$ does not divide the conductor, $p$ does not divide the  order and $f<10$ such that $\lambda>0$ (resp. $\lambda=1$ in the trivial zero case).

In [10]:
p=3
print("p=",p)
for k in range(2,100):
            H = DirichletGroup(k)
            for i in range(H.order()):
                chi=H[i]
                n = chi.order() 
                if gcd(n,p)==1:
                    f = mod(p,n).multiplicative_order() # degree of field extension of Qp containing $n$-th root of unity    
                chi0 = chi.change_ring(CyclotomicField(n)) 
                if chi0.is_primitive() and gcd(chi0.conductor(),p)==1 and gcd(n,p)==1 and f<10:
                    if (chi0.is_odd()):
                        odd=1
                        oddtext="odd"
                    else:
                        odd=0
                        oddtext="even"

                    for twist in range(odd,p-1,2):
                        
                        la=lambda_invariant(chi0,p,twist) 

                        if twist==1 and chi0(p)==1:
                            triv=", trivial zero"  # trivial zero
                            threshold=1 
                        else:
                            triv=""
                            threshold=0

                        if la>threshold:    # la>0; la>1 for trivial zero
                       
                    
                            print("lambda=",la,",",oddtext, chi0,",","twist=",twist,",","order=", n, ",","f=",f, triv)   

p= 3
lambda= 1 , even Dirichlet character modulo 29 of conductor 29 mapping 2 |--> -1 , twist= 0 , order= 2 , f= 1 
lambda= 1 , odd Dirichlet character modulo 31 of conductor 31 mapping 3 |--> -1 , twist= 1 , order= 2 , f= 1 
lambda= 1 , even Dirichlet character modulo 32 of conductor 32 mapping 31 |--> 1, 5 |--> -zeta8 , twist= 0 , order= 8 , f= 2 
lambda= 1 , even Dirichlet character modulo 32 of conductor 32 mapping 31 |--> 1, 5 |--> -zeta8^3 , twist= 0 , order= 8 , f= 2 
lambda= 2 , odd Dirichlet character modulo 35 of conductor 35 mapping 22 |--> -1, 31 |--> -1 , twist= 1 , order= 2 , f= 1 , trivial zero
lambda= 2 , odd Dirichlet character modulo 47 of conductor 47 mapping 5 |--> -1 , twist= 1 , order= 2 , f= 1 , trivial zero
lambda= 2 , odd Dirichlet character modulo 56 of conductor 56 mapping 15 |--> 1, 29 |--> -1, 17 |--> -1 , twist= 1 , order= 2 , f= 1 , trivial zero
lambda= 2 , even Dirichlet character modulo 77 of conductor 77 mapping 45 |--> -1, 57 |--> -1 , twist= 0 , orde

### More data for $p=3,5,7$ and conductor $<1000$ can be found in separate files.

In [11]:
# Example of a large lambda-invariant
H = DirichletGroup(4*53301/3, base_ring=CyclotomicField(2)) # quadratic character of conductor 

In [20]:
lambda_invariant(H[7],3,0) # lambda=11

11